In [1]:
from pyathena import connect
import pandas as pd

conn = connect(aws_access_key_id='YOUR_ACCESS_KEY_ID'
               aws_secret_access_key='YOUR_SECRET_ACCESS_KEY',
               s3_staging_dir='s3://athena-results2/cleansing/',
               region_name='us-east-1')
df = pd.read_sql("SELECT * FROM time_series.clean_log2 limit 100", conn)
print(df.head())

   processing_time  record_timestamp  ainat160_value_d  \
0    1565704205420     1565704040439          0.930476   
1    1565704205420     1565704041440          0.930476   
2    1565704205420     1565704042439          0.870785   
3    1565704205420     1565704043438          0.912923   
4    1565704205420     1565704044439          0.937501   

   ainbsc001_status1_value_d  ainbsc001_status2_value_d  ainct430_value_d  \
0                        0.0                        0.0          0.108848   
1                        0.0                        0.0          0.109200   
2                        0.0                        0.0          0.108848   
3                        0.0                        0.0          0.108146   
4                        0.0                        0.0          0.108848   

   ainct450_value_d  ainextpowmeas_value_d  ainfan150control_value_d  \
0          0.214888           -1500.140503                    4919.0   
1          0.214536           -1500.140503  

In [2]:
database = 'time_series'
table = 'clean_log2'

query_1 = """ SELECT 
            current_date AS today,
            date(date_parse(record_timestamp_new,'%Y-%m-%d %H:%i:%s')) AS date, 
            count(alarm) AS count_state
            
        FROM time_series.clean_log2
        WHERE date(date_parse(record_timestamp_new,'%Y-%m-%d %H:%i:%s'))
                    BETWEEN current_date - interval '120' day
                    AND current_date
              AND alarm >=150
        GROUP BY  date(date_parse(record_timestamp_new,'%Y-%m-%d %H:%i:%s')),alarm
        ORDER by date,alarm;""".format(database, table)

In [3]:
df = pd.read_sql(query_1, conn)
df.head()

,today,date,count_pstate
0,2019-12-10,2019-08-12,86400
1,2019-12-10,2019-08-13,86400
2,2019-12-10,2019-08-14,86400
3,2019-12-10,2019-08-15,86400
4,2019-12-10,2019-08-16,86460
